Partie 1: imports   

In [1]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

 Partie 2 : Charger le modèle IA + les classes

In [2]:
# load modèle
model_path = "../ia_model/saved_model/bird_classifier.h5"
model = load_model(model_path)

# load name d'espèces (dans le bon ordre)
with open("../ia_model/saved_model/classes.txt", "r") as f:
    class_names = [line.strip() for line in f.readlines()]


Partie 3 : Fonction pour convertir un fichier audio en spectrogramme (image)

In [3]:
def audio_to_spectrogram_image(audio_path, img_size=(128, 128)):
    y, sr = librosa.load(audio_path, sr=None)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mel_db = librosa.power_to_db(mel, ref=np.max)

    # Sauvegarde temporaire d'une image (ou conversion directe en array)
    fig = plt.figure(figsize=(4, 4))
    librosa.display.specshow(mel_db, sr=sr, x_axis=None, y_axis=None, cmap='magma')
    plt.axis('off')
    plt.tight_layout(pad=0)

    # Sauvegarde temporaire
    temp_img = "temp.png"
    plt.savefig(temp_img, bbox_inches='tight', pad_inches=0)
    plt.close()

    # Recharger l'image, la redimensionner et la convertir en tableau
    img = load_img(temp_img, target_size=img_size)
    img_array = img_to_array(img) / 255.0  # Normalisation
    os.remove(temp_img)
    
    return np.expand_dims(img_array, axis=0)


Partie 4 : Prédiction d’un fichier audio

In [4]:
# path vers un fichier audio à tester
audio_file = "../bird_audio/<nom_dossier>/<fichier_test.wav>"  # à modifier

# Conversion
image_input = audio_to_spectrogram_image(audio_file)

# Prédiction
prediction = model.predict(image_input)
predicted_index = np.argmax(prediction)
predicted_label = class_names[predicted_index]
confidence = prediction[0][predicted_index] * 100

print(f"🐦 Espèce prédite : {predicted_label} ({confidence:.2f}%)")


C:\Users\willi\AppData\Local\Temp\ipykernel_14248\3502792856.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=None)
c:\Users\willi\Documents\Ecole_ipssi_MIA4\avril_mai_2025\tp_groupe\.conda\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


OSError: [Errno 22] Invalid argument: '../bird_audio/<nom_dossier>/<fichier_test.wav>'